<a href="https://colab.research.google.com/github/liampearson/Google_Mobility_Data/blob/main/mobility_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from os.path import join
import math

from google.colab import drive

# authentication for google sheets
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
write_directory = '/content/drive/My Drive/covid19data/'

In [ ]:
# region csv is located at: https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip

#download the zip file
!wget https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip

#unzip the file
!unzip Region_Mobility_Report_CSVs.zip

#get the AU file
REGION = "AU"
df2020 = pd.read_csv("2020_"+REGION+"_Region_Mobility_Report.csv")
df2021 = pd.read_csv("2021_"+REGION+"_Region_Mobility_Report.csv")
df = df2020.append(df2021)

df.head()

--2021-08-03 22:30:55--  https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip
Resolving www.gstatic.com (www.gstatic.com)... 172.217.204.94, 2607:f8b0:400c:c13::5e
Connecting to www.gstatic.com (www.gstatic.com)|172.217.204.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53778895 (51M) [application/zip]
Saving to: ‘Region_Mobility_Report_CSVs.zip’

Region_Mobility_Rep 100%[===================>]  51.29M  7.15MB/s    in 7.3s    

2021-08-03 22:31:03 (6.98 MB/s) - ‘Region_Mobility_Report_CSVs.zip’ saved [53778895/53778895]

Archive:  Region_Mobility_Report_CSVs.zip
  inflating: 2020_AE_Region_Mobility_Report.csv  
  inflating: 2020_AF_Region_Mobility_Report.csv  
  inflating: 2020_AG_Region_Mobility_Report.csv  
  inflating: 2020_AO_Region_Mobility_Report.csv  
  inflating: 2020_AR_Region_Mobility_Report.csv  
  inflating: 2020_AT_Region_Mobility_Report.csv  
  inflating: 2020_AU_Region_Mobility_Report.csv  
  inflating: 2020_AW_Region_Mobi

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-15,4.0,3.0,-2.0,3.0,3.0,0.0
1,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-16,3.0,5.0,9.0,3.0,-1.0,0.0
2,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-17,-1.0,0.0,-6.0,7.0,17.0,-2.0
3,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-18,-3.0,-2.0,-13.0,7.0,14.0,-1.0
4,AU,Australia,NaN,NaN,NaN,NaN,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,2020-02-19,-1.0,-1.0,-6.0,8.0,13.0,-1.0


In [ ]:
#inspect the data (see first 5 rows)
df.columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [ ]:
#notes for AU:
#df.sub_region_1 = State = [, 'Australian Capital Territory', 'New South Wales', 'Northern Territory', 'Queensland', 'South Australia', 'Tasmania', 'Victoria', 'Western Australia']
#df.sub_region_2 = councils
#df.metro_area = null
#df.iso_3166_2_code.unique() = [nan, 'AU-ACT', 'AU-NSW', 'AU-NT', 'AU-QLD', 'AU-SA', 'AU-TAS','AU-VIC', 'AU-WA']
#df[df.iso_3166_2_code=='AU-NSW'].date.tail()
#df[df.date==max(df.date)]

## Get place_id lookup (equivalent of index_v)
Google assigned a unique `place_id` for each Country / sub_region_1 / sub_region_2...etc

For each unique place_id, we will get 3 fields:
- Country / sub_region_1 / sub_region_2

In [ ]:
# initialise an empty dataframe for the place_id lookup
place_id_lookup = pd.DataFrame()

for placeid in df.place_id.unique():
  #ensure there is only one country_region for this placeid
  if (len(df[df['place_id']==placeid]['country_region'].unique())!=1):
    print("ERROR!!!")
  country_region = df[df['place_id']==placeid]['country_region'].unique()[0]

  #ensure there is only one iso_3166_2_code for this placeid
  if (len(df[df['place_id']==placeid]['iso_3166_2_code'].unique())!=1):
    print("ERROR!!!")
  iso_3166_2_code = df[df['place_id']==placeid]['iso_3166_2_code'].unique()[0]

  #ensure there is only one sub_region_1 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_1'].unique())!=1):
    print("ERROR!!!")
  sub_region_1 = df[df['place_id']==placeid]['sub_region_1'].unique()[0]

  #ensure there is only one sub_region_2 for this placeid
  if (len(df[df['place_id']==placeid]['sub_region_2'].unique())!=1):
    print("ERROR!!!")
  sub_region_2 = df[df['place_id']==placeid]['sub_region_2'].unique()[0]


  new_row = {'place_id'       : placeid,
             'iso_3166_2_code': iso_3166_2_code,
             'country_region' : country_region, 
             'sub_region_1'   : sub_region_1, 
             'sub_region_2'   : sub_region_2
            }

  #append row to the dataframe
  place_id_lookup = place_id_lookup.append(new_row, ignore_index=True)

#inspect the place_id_lookup
place_id_lookup

,country_region,iso_3166_2_code,place_id,sub_region_1,sub_region_2
0,Australia,NaN,ChIJ38WHZwf9KysRUhNblaFnglM,NaN,NaN
1,Australia,AU-ACT,ChIJSxCboN9MFmsRA3huXDhEWOc,Australian Capital Territory,NaN
2,Australia,AU-NSW,ChIJDUte93TLDWsRLZ_EIhGvgBc,New South Wales,NaN
3,Australia,NaN,ChIJPXdpvMSnn2sRamkBGce1LIM,New South Wales,Armidale Dumaresq Council
4,Australia,NaN,ChIJGSMhcwKCkGsRWLKq2JtdcD0,New South Wales,Ballina Shire Council
...,...,...,...,...,...
296,Australia,NaN,ChIJywG3noSlMioRCz8TieOMH0Q,Western Australia,Town of Cambridge
297,Australia,NaN,ChIJzy2ctjekMioRCcmcWIaa8IQ,Western Australia,Town of Claremont
298,Australia,NaN,ChIJxf2tdx-kMioRJOZpQ6GJ2hQ,Western Australia,Town of Cottesloe
299,Australia,NaN,ChIJk9DO_Yoq9SsRX_KCepEBXlA,Western Australia,Town of Port Hedland


In [ ]:
def get_timestamp():
  from datetime import datetime, timezone

  #get sydney timezone
  import pytz
  sydney_tz = pytz.timezone('Australia/Sydney')
  now = datetime.now(sydney_tz)

  # format to dd/mm/YY H:M:S with timezone
  dt_string = now.strftime("%d/%m/%Y %H:%M:%S %z")
  return dt_string

get_timestamp()

'04/08/2021 08:31:31 +1000'

In [ ]:
def write_to_gsheet(workbook_title, worksheet_title, data):
  # open spreadsheet
  sh = gc.open(workbook_title)

  #select an existing worksheet (tab)
  worksheet = sh.worksheet(worksheet_title)

  #delete that worksheet
  sh.del_worksheet(worksheet)

  #recreate worksheet (tab) and populate with dataframe
  worksheet = sh.add_worksheet(title=worksheet_title, rows="1000", cols="200")
  set_with_dataframe(worksheet, data, include_index=True)

1. Pivot the data
2. Move the 'Australia' label down to the other levels
3. Move the State labels down to the sub_region_2



 

In [ ]:
CATEGORIES = ['retail_and_recreation_percent_change_from_baseline',
              'grocery_and_pharmacy_percent_change_from_baseline',
              'parks_percent_change_from_baseline',
              'transit_stations_percent_change_from_baseline',
              'workplaces_percent_change_from_baseline',
              'residential_percent_change_from_baseline'
            ]

# update the fields for AU (sub regions are null for Country)
# want unique state values excluding null
df.loc[(df.sub_region_1.isnull()) & (df.sub_region_2.isnull()) & (df.iso_3166_2_code.isnull()) & (df['country_region']=='Australia'), 'sub_region_1'] = 'AUS'
df.loc[(df.sub_region_2.isnull()) & (df.iso_3166_2_code.isnull()) & (df['country_region']=='Australia'), 'sub_region_2'] = 'AUS'

# move the iso_3166_2_code (i.e. state) into sub_region_2
# want unique state values excluding null
for state in df.iso_3166_2_code.copy().dropna().unique():
  df.loc[df['iso_3166_2_code']==state, 'sub_region_2'] = state[3:]


cols = ['sub_region_1', 'sub_region_2']


# for each category, pivot on that category
for category in CATEGORIES:
  print("processing {}".format(category))
  mobility_data = df.pivot(index = 'date', columns = cols, values=category)

  #get rolling average
  rolling_mean_df = mobility_data.rolling(7).mean()

  #add timestamp
  mobility_data['last_updated'] = get_timestamp()
 # mobility_data_pivoted['last_updated'] = get_timestamp()
  rolling_mean_df['last_updated'] = get_timestamp()

  #write to googlesheets
  write_to_gsheet(workbook_title='mobility_data',
                worksheet_title = category.split('_')[0],
                data = mobility_data)
  
  write_to_gsheet(workbook_title='mobility_data_rolling_7day_average',
                worksheet_title = category.split('_')[0],
                data = rolling_mean_df)

processing retail_and_recreation_percent_change_from_baseline
processing grocery_and_pharmacy_percent_change_from_baseline
processing parks_percent_change_from_baseline
processing transit_stations_percent_change_from_baseline
processing workplaces_percent_change_from_baseline
processing residential_percent_change_from_baseline


In [ ]:
print("SUCCESS - script has run as of {}".format(get_timestamp()))

SUCCESS - script has run as of 04/08/2021 08:32:47 +1000


In [ ]:
#get LGA data just before lockdown
#test = df[df.date>='2021-06-01']
#test = test[['date', 'sub_region_1', 'sub_region_2']+CATEGORIES]

#write_to_gsheet(workbook_title='TEST_mobility_data_pivoted',
#              worksheet_title = 'all_categories',
#              data = test)